In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-19"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7904,2024-07-19,Bolívia Libobasquet,21:30,Carl A-Z,Tarija Basquet,3.02,1.35,164.5,1.85,1.85,0.0,0.00,0.00,COJQpqbE,0.331126,0.740741,0.540541,0.540541,0.071867,211.618,78.842411,0.372569,0.6,1.341641,2.236068,242.64,2.896,1.471149,0.507994,-16.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,72,90,3.37,1.29,188.838,0.000,0.540443,0.000000,NaN,-1.99,-0.398,-5.075377,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7905,2024-07-19,Brasil Lbf Feminina,21:00,SESI Araraquara F,Sampaio F,1.12,5.56,128.5,1.84,1.86,0.0,0.00,0.00,tr0oPG4C,0.892857,0.179856,0.543478,0.537634,0.072713,113.580,40.754589,0.358818,1.8,1.643168,0.912871,116.61,1.746,0.598899,0.343012,27.0,81.832,18.654944,0.227966,1.8,1.643168,0.912871,98.88,1.372,0.418234,0.304836,11.0,69,48,1.69,2.06,97.432,479.428,0.939986,0.007644,NaN,0.35,0.070,1.714286,0.838063,0.9,0.061937,19.30,3.860,1.181347,0.707743,0.8,0.092257
7906,2024-07-19,América Campeonato Mundial,23:00,El Salvador,Costa Rica,2.78,1.42,144.5,1.82,1.90,0.0,0.00,0.00,U7lbZqYH,0.359712,0.704225,0.549451,0.526316,0.063938,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,70,94,1.60,1.09,0.000,0.000,0.457936,0.030413,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7907,2024-07-19,América Campeonato Mundial,20:30,Honduras,Guatemala,1.29,3.46,139.5,1.81,1.91,0.0,0.00,0.00,Msojy635,0.775194,0.289017,0.552486,0.523560,0.064211,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,62,47,2.27,6.85,0.000,0.000,0.646072,0.038016,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7908,2024-07-19,República Dominicana Lnb,21:00,Marineros,Reales de la Vega,1.90,1.82,172.5,1.87,1.79,3.5,1.55,1.47,IwteMxwS,0.526316,0.549451,0.534759,0.558659,0.075766,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,136.084,53.394287,0.392363,1.8,1.643168,0.912871,107.07,1.532,0.430836,0.281225,7.0,82,83,1.81,1.29,0.000,0.000,0.030413,0.030912,0.037463,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7909,2024-07-19,República Dominicana Lnb,21:00,Metros de Santiago,Leones,1.66,2.13,166.5,1.87,1.79,0.0,0.00,0.00,0YBN5Lk3,0.602410,0.469484,0.534759,0.558659,0.071893,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,92,78,3.37,1.91,0.000,0.000,0.175377,0.030912,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
7910,2024-07-19,Eua Nba,17:00,Phoenix Suns,Milwaukee Bucks,1.63,2.26,181.5,1.89,1.89,-3.5,1.93,2.83,GEwxrlal,0.613497,0.442478,0.529101,0.529101,0.055975,222.922,48.598659,0.218007,0.6,1.341641,2.236068,171.00,2.368,0.881884,0.372417,-43.0,258.126,76.090088,0.294779,0.6,1.341641,2.236068,259.96,2.678,0.674218,0.251762,-52.0,100,97,1.71,2.68,196.404,322.030,0.229037,0.000000,0.267393,-5.00,-1.000,-0.630000,0.608075,0.4,-0.208075,-5.00,-1.000,-1.260000,0.451390,0.1,-0.351390
7911,2024-07-19,Nova Zelândia Nbl,04:30,Wellington Saints,Franklin Bul

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

Sem jogos
